# MUSIC GENERATION
## PART II

In [1]:
from music21 import *
import numpy as np

In [2]:
import pickle 

In [3]:
%matplotlib inline

In [4]:
#loading the input network
with open("network_input.pkl", "rb") as v:
    network_input = pickle.load(v)

In [5]:
#loading a dictionary to convert the integers back into notes and chords
with open("int_to_note.pkl", "rb") as t:
    int_to_note = pickle.load(t)

In [6]:
n_vocab = len(int_to_note)

### Loading model architecture

In [7]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint

In [8]:
model = Sequential()

In [9]:
model.add(LSTM( 500, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
model.add(Dropout(0.2))

In [10]:
model.add(LSTM(500, return_sequences=True))
model.add(Dropout(0.2))

In [11]:
model.add(Dense(500))

In [12]:
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))

In [13]:
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.2))

In [14]:
model.add(Dense(n_vocab))

In [15]:
model.add(Activation('softmax'))

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
#loading the trained model
model.load_weights('Model_Weights.hdf5')

In [18]:
#Finging a random note or chord to start from
start = np.random.randint(0, len(network_input)-1)

In [19]:
pattern = network_input[start]
prediction_output = []

In [20]:
#Prediction the next note storing it in the array and predicting the note with new array formed
for note_index in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction = model.predict(prediction_input)
    index = np.where(prediction == np.amax(prediction))
    result = int_to_note[index[1][0]]
    prediction_output.append(result)
    pattern = np.append(pattern, index[1]/n_vocab)
    pattern = pattern[1:len(pattern)]

In [21]:
offset = 0
output_notes = []

In [22]:
from music21 import *

In [23]:
# converting the notes and chords into music21 object
for pattern in prediction_output:
    # chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    
    offset += 0.5

In [24]:
#Converting note and chord objects into a stream object and creating and midi file
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='Generated_Melody.mid')

'Generated_Melody.mid'

In [25]:
# creating a score object of stream to hear the final output
m = converter.parse('Generated_Melody.mid')

### Results

In [26]:
m.show('midi')